In [1]:
# Clean this up, many are unused

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import time
from sklearn.model_selection import RandomizedSearchCV

df_raw = pd.read_csv('BankChurners.csv')
y = df_raw['Attrition_Flag'].to_numpy()

In [2]:
cols = df_raw.describe().columns
cols = cols.tolist()
dfx = df_raw[cols[1:-2]]
X_num = dfx.to_numpy()
print(str(X_num.shape[1]) + ' Numerical Only')

# categorical, Gender
df_gender = pd.get_dummies(df_raw['Gender'])
genF = df_gender['F'].to_numpy()
Xf = np.append(X_num,genF.reshape((10127,1)),axis=1)
print(str(Xf.shape[1]) + ' with Sex')

# print(Xf.shape)
df_edu = pd.get_dummies(df_raw['Education_Level'])
edu = df_edu.to_numpy()
# print(edu.shape)
Xed = np.append(Xf,edu,axis=1)
print(str(Xed.shape[1]) + ' with Education, should add 7 to 22')

# categorical, Marriage
# Put 4 columns because is divorced more likely than single to use this card, I dont think so
mar = df_raw['Marital_Status'].unique()
# print(mar)
df_mar = pd.get_dummies(df_raw['Marital_Status'])
# df_mar.head()
mar = df_mar.to_numpy()
# print(mar.shape)
Xmar = np.append(Xed,mar,axis=1)
print(str(Xmar.shape[1]) + ' with Marriage, should add 4 to 26')

# categorical, Income
# Put 6 columns because i am not sure if this should be ordinal but I am saying not.
inc = df_raw['Income_Category'].unique()
df_inc = pd.get_dummies(df_raw['Income_Category'])
df_inc.head()
inc = df_inc.to_numpy()
Xinc = np.append(Xmar,inc,axis=1)
print(str(Xinc.shape[1]) + ' with Income, should add 6 to 32')

# categorical, Card Category
# Put 6 columns because i am not sure if this should be ordinal but I am saying not, dont know what the order is.
cc = df_raw['Card_Category'].unique()
df_cc = pd.get_dummies(df_raw['Card_Category'])
df_cc.tail()
cc = df_cc.to_numpy()
Xcc = np.append(Xinc,cc,axis=1)
print(str(Xcc.shape[1]) + ' with Card Color, should add 4 to 36')


# Final X, just rename
X = Xcc
print(X.shape)

14 Numerical Only
15 with Sex
22 with Education, should add 7 to 22
26 with Marriage, should add 4 to 26
32 with Income, should add 6 to 32
36 with Card Color, should add 4 to 36
(10127, 36)


I will make a custom scorer for my hyperparameterizations to come.

In [3]:
from sklearn.metrics import make_scorer

def custom_recall_score(y_test, y_pred):
    return metrics.recall_score(y_test,y_pred, average=None)[0]

c_score = make_scorer(custom_recall_score)

<h3>First Model is the Random Forest - 62<h3>

In [4]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=0)
scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

clf = RandomForestClassifier(max_depth=8, random_state=0) # <-- max depth could be the problem
clf.fit(X_train, y_train) #< fit on the train, score on the test
y_pred = clf.predict(X_test)
print("Recall:",metrics.recall_score(y_test,y_pred, average=None))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))

Recall: [0.6171875  0.98696462]
[[ 237  147]
 [  28 2120]]
                   precision    recall  f1-score   support

Attrited Customer       0.89      0.62      0.73       384
Existing Customer       0.94      0.99      0.96      2148

         accuracy                           0.93      2532
        macro avg       0.91      0.80      0.85      2532
     weighted avg       0.93      0.93      0.93      2532



<h3>Second Model is the Hyperparameterized Random Forest - 79<h3>

In [5]:
clf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 8,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 0,
 'verbose': 0,
 'warm_start': False}

In [6]:
#--------------------------------- WARNING -- 14 Min run time --------------------------

# Try Minimal, this one was garbage
param_grid_rf = {
    'bootstrap': [True],
    'max_depth': [80, 110],
    'max_features': ['auto', 3],
    'min_samples_leaf': [1,3],
    'min_samples_split': [2, 8],
    'n_estimators': [100, 1000]
}

tic = time.perf_counter() 
rf_model_grid = GridSearchCV(estimator=RandomForestClassifier(),
                            param_grid=param_grid_rf, 
                            scoring=c_score,
                            cv = 5,
                            verbose=0)# verbose = 3 to read output

rf_model_grid.fit(X_train, y_train)
print(rf_model_grid.best_estimator_)


toc = time.perf_counter()
print(str( (toc - tic)/60) + ' minutes')

RandomForestClassifier(max_depth=110, n_estimators=1000)
13.535258480000001 minutes


In [13]:
# From Grid Search
# clf_h = RandomForestClassifier(max_depth=110,
#                                n_estimators=1000)

# This one is better for some reason
clf_h = RandomForestClassifier(max_depth=110,
                               random_state=13)


y_pred = clf_h.fit(X_train, y_train).predict(X_test)

print("Recall:",metrics.recall_score(y_test,y_pred, average=None))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))

Recall: [0.7890625  0.98696462]
[[ 303   81]
 [  28 2120]]
                   precision    recall  f1-score   support

Attrited Customer       0.92      0.79      0.85       384
Existing Customer       0.96      0.99      0.97      2148

         accuracy                           0.96      2532
        macro avg       0.94      0.89      0.91      2532
     weighted avg       0.96      0.96      0.96      2532



In [14]:
#--------------------------------------------  2 Min run time --------------------------

param_dist = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [2, 3, 4, 5, 8],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

tic = time.perf_counter()
rf_random_search = RandomizedSearchCV(estimator=RandomForestClassifier(),
                                   param_distributions=param_dist,
                                   scoring=c_score,
                                   cv = 5,
                                   verbose=0,
                                   n_iter=10)


rf_random_search.fit(X_train, y_train)
print(rf_random_search.best_estimator_)
print(rf_random_search.best_score_)

toc = time.perf_counter()
print(str( (toc - tic)/60) + ' minutes')

RandomForestClassifier(max_depth=80, max_features=3, min_samples_leaf=3,
                       min_samples_split=8, n_estimators=300)
0.6347778209612643
2.1551986700000003 minutes


In [16]:
# for some reason this gives a different answer (59) from above (63)
# clf_rand = RandomForestClassifier(max_depth=90, max_features=3, min_samples_leaf=2,
#                                   min_samples_split=8, n_estimators=1000,
#                                   random_state=43)

# I just grabbed this to see.
clf_rand = RandomForestClassifier(max_depth=110,
                                  max_features=3,
                                  min_samples_leaf=2,
                                  min_samples_split=10,
                                  n_estimators=100,
                                  random_state=13)

y_pred = clf_rand.fit(X_train, y_train).predict(X_test)

print("Recall:",metrics.recall_score(y_test,y_pred, average=None))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))

Recall: [0.6015625  0.99022346]
[[ 231  153]
 [  21 2127]]
                   precision    recall  f1-score   support

Attrited Customer       0.92      0.60      0.73       384
Existing Customer       0.93      0.99      0.96      2148

         accuracy                           0.93      2532
        macro avg       0.92      0.80      0.84      2532
     weighted avg       0.93      0.93      0.93      2532



<h3>The third model is the Naive Bayes Classifier - 63<h3>

In [17]:
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)

print("Recall:",metrics.recall_score(y_test,y_pred, average=None))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))

Recall: [0.6328125  0.92225326]
[[ 243  141]
 [ 167 1981]]
                   precision    recall  f1-score   support

Attrited Customer       0.59      0.63      0.61       384
Existing Customer       0.93      0.92      0.93      2148

         accuracy                           0.88      2532
        macro avg       0.76      0.78      0.77      2532
     weighted avg       0.88      0.88      0.88      2532



<h3>The fourth model is the Hyperparameterized Naive Bayes Classifier - 63<h3>

In [18]:
gnb.get_params()

{'priors': None, 'var_smoothing': 1e-09}

In [19]:
param_grid_nb = {'var_smoothing': np.logspace(0,-9, num=100)}

nbModel_grid = GridSearchCV(estimator=GaussianNB(), 
                            param_grid=param_grid_nb, 
                            scoring=c_score,
                            cv = 10,
                            verbose=0)# verbose = 3 to read output
tic = time.perf_counter()

nbModel_grid.fit(X_train, y_train)
print(nbModel_grid.best_estimator_)

toc = time.perf_counter()
print(str( (toc - tic)/60) + ' minutes')

GaussianNB(var_smoothing=0.001873817422860383)
0.3943668549999984 minutes


In [21]:
#gnbh = GaussianNB(var_smoothing=0.000000001)  <--- Defaults
gnbh = GaussianNB(var_smoothing=0.001873817422860383)
y_pred = gnbh.fit(X_train, y_train).predict(X_test)

print("Recall:",metrics.recall_score(y_test,y_pred, average=None))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))

Recall: [0.6328125  0.92225326]
[[ 243  141]
 [ 167 1981]]
                   precision    recall  f1-score   support

Attrited Customer       0.59      0.63      0.61       384
Existing Customer       0.93      0.92      0.93      2148

         accuracy                           0.88      2532
        macro avg       0.76      0.78      0.77      2532
     weighted avg       0.88      0.88      0.88      2532



<h3>Fifth Model is the Logistic Regression - 60<h3>

In [22]:
logreg = LogisticRegression()
y_pred = logreg.fit(X_train,y_train).predict(X_test)

print("Recall:",metrics.recall_score(y_test,y_pred, average=None))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))

Recall: [0.59895833 0.95949721]
[[ 230  154]
 [  87 2061]]
                   precision    recall  f1-score   support

Attrited Customer       0.73      0.60      0.66       384
Existing Customer       0.93      0.96      0.94      2148

         accuracy                           0.90      2532
        macro avg       0.83      0.78      0.80      2532
     weighted avg       0.90      0.90      0.90      2532



<h3>Fifth is the Logistic Regression with Hyperparameterization - 60<h3>

In [23]:
# parameters = {
#     'penalty' : ['none'], 
#     'C'       : np.logspace(-3,3,7),
#     'solver'  : ['newton-cg', 'lbfgs', 'liblinear'],
#     'max_iter': [1000000]
# }

parameters = {
    'C'       : np.logspace(-3,3,7),
    'solver'  : ['newton-cg', 'lbfgs', 'liblinear']
}



logreg = LogisticRegression()
clf = GridSearchCV(logreg,                    # model
                   param_grid = parameters,   # hyperparameters
                   scoring=c_score,           # metric for scoring ---> used to be 'recall', does not work
                   cv = 10,                   # something with cross validation, higher gets better score
                   verbose=0) # verbose = 3 to read output

clf.fit(X_train,y_train)

GridSearchCV(cv=10, estimator=LogisticRegression(),
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'solver': ['newton-cg', 'lbfgs', 'liblinear']},
             scoring=make_scorer(custom_recall_score))

In [24]:
print("Tuned Hyperparameters :", clf.best_params_)
print("Recall :",clf.best_score_)

Tuned Hyperparameters : {'C': 10.0, 'solver': 'newton-cg'}
Recall : 0.5993677419354839


In [25]:
#logreg = LogisticRegression(C = 1.0, solver = 'lbfgs') # <---Defaults
logreg = LogisticRegression(C = 10.0, solver = 'newton-cg')
y_pred = logreg.fit(X_train,y_train).predict(X_test)
# y_pred = logreg.predict(X_test)
print("Recall:",metrics.recall_score(y_test,y_pred, average=None))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))

Recall: [0.59895833 0.95810056]
[[ 230  154]
 [  90 2058]]
                   precision    recall  f1-score   support

Attrited Customer       0.72      0.60      0.65       384
Existing Customer       0.93      0.96      0.94      2148

         accuracy                           0.90      2532
        macro avg       0.82      0.78      0.80      2532
     weighted avg       0.90      0.90      0.90      2532



<h3>Questions</h3>
<ol>
  <li>Random Forest - Why does the grid search not give the best answer?</li>
  <li>Why does the random_state variable have so much control over outcomes?</li>  
  <li>Others - Why did hyperparameterization not improve the model?</li>
  <li>How do I determine the range for parameters of a model, I am copying and guessing?</li>
</ol>